In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/test-image/download.png
/kaggle/input/fakereal-logo-detection-dataset/file_mapping.csv
/kaggle/input/fakereal-logo-detection-dataset/Logos.txt
/kaggle/input/fakereal-logo-detection-dataset/output/Hp/000004.jpg
/kaggle/input/fakereal-logo-detection-dataset/output/Hp/000003.jpg
/kaggle/input/fakereal-logo-detection-dataset/output/Hp/000002.jpg
/kaggle/input/fakereal-logo-detection-dataset/output/Hp/000001.jpg
/kaggle/input/fakereal-logo-detection-dataset/output/Hp/000005.jpg
/kaggle/input/fakereal-logo-detection-dataset/output/Fila/000004.jpg
/kaggle/input/fakereal-logo-detection-dataset/output/Fila/000003.jpg
/kaggle/input/fakereal-logo-detection-dataset/output/Fila/000002.jpg
/kaggle/input/fakereal-logo-detection-dataset/output/Fila/000001.jpg
/kaggle/input/fakereal-logo-detection-dataset/output/Fila/000005.jpg
/kaggle/input/fakereal-logo-detection-dataset/output/Twitter/000004.jpg
/kaggle/input/fakereal-logo-detection-dataset/output/Twitter/000003.jpg
/kaggle/input/faker

**Importing Dependencies**

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

**Modifying CSV - Data-Preprocessing**

In [3]:
import csv

# Open the CSV file
with open('/kaggle/input/fakereal-logo-detection-dataset/file_mapping.csv', 'r') as file:
    reader = csv.reader(file)
    rows = list(reader)

# Iterate over the rows and replace backslashes with forward slashes
for row in rows:
    row[0] = row[0].replace('\\', '/')

# Save the modified CSV file
with open('modified_csv_file.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(rows)

**Data-Loader**

In [4]:
class LogoDataset(Dataset):
    def __init__(self, csv_path, root_dir, transform=None):
        self.data = pd.read_csv(csv_path)
        self.root_dir = root_dir
        self.transform = transform

        # self.vectorizer = CountVectorizer()

        # Fit the vectorizer on taglines
        # self.data['Tagline'].fillna('', inplace=True)

        # self.vectorizer = CountVectorizer()
        # self.vectorizer.fit(self.data['Tagline'])
                
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.data['Filename'][idx])
        image = Image.open(img_path).convert('RGB')
        label_name = self.data['Label'][idx]
        # tagline = self.data['Tagline'][idx]
        
        if label_name == "Genuine":
            label = 0
        else:
            label = 1
            
        if self.transform:
            image = self.transform(image)
        
        # tagline_vector = self.vectorizer.transform([tagline]).toarray()
        # tagline_tensor = torch.tensor(tagline_vector, dtype=torch.float32).squeeze()
            
        return image, label #,tagline_tensor

In [5]:
dataset_path = '/kaggle/input/fakereal-logo-detection-dataset'
csv_file = '/kaggle/working/modified_csv_file.csv'

**Data Augmentation - Applying Tranformation**

In [6]:
image_transforms = transforms.Compose([
    transforms.Resize((70, 70)),
    transforms.RandomCrop(64),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406],
    #                      std=[0.229, 0.224, 0.225])
])

**Loading Data**

In [7]:
logo_dataset = LogoDataset(csv_file, dataset_path, transform=image_transforms)

In [8]:
batch_size = 32
dataloader = DataLoader(logo_dataset, batch_size=batch_size, shuffle=True)

In [9]:
model = models.resnet50(pretrained=True)
num_classes = 2  # Adjust this based on the number of classes in your dataset
in_features = model.fc.in_features
model.fc = torch.nn.Linear(in_features, num_classes)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 238MB/s]


**Initializing Loss and Fine tuning model**

In [10]:
# embedding_dim = 2048  # Adjust the dimensionality of the embedding to match the model's fc layer
# tagline_embedding = torch.nn.Linear(len(logo_dataset.vectorizer.vocabulary_), embedding_dim)

model.fc = torch.nn.Linear(in_features, num_classes)

# Define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Set the device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# tagline_embedding = tagline_embedding.to(device)

**Model training till 25 epochs**

In [11]:
num_epochs = 25
save_interval = 5

for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")
    if (epoch + 1) % save_interval == 0:
            # Save the trained model
            torch.save(model.state_dict(), f'logo_classification_model_epoch{epoch+1}.pth')

Epoch [1/25], Loss: 0.9087
Epoch [2/25], Loss: 0.5350
Epoch [3/25], Loss: 0.4442
Epoch [4/25], Loss: 0.3651
Epoch [5/25], Loss: 0.3374
Epoch [6/25], Loss: 0.3742
Epoch [7/25], Loss: 0.2506
Epoch [8/25], Loss: 0.3070
Epoch [9/25], Loss: 0.3307
Epoch [10/25], Loss: 0.2754
Epoch [11/25], Loss: 0.3104
Epoch [12/25], Loss: 0.3001
Epoch [13/25], Loss: 0.4368
Epoch [14/25], Loss: 0.3452
Epoch [15/25], Loss: 0.3521
Epoch [16/25], Loss: 0.3281
Epoch [17/25], Loss: 0.2962
Epoch [18/25], Loss: 0.2439
Epoch [19/25], Loss: 0.2233
Epoch [20/25], Loss: 0.2409
Epoch [21/25], Loss: 0.2214
Epoch [22/25], Loss: 0.2017
Epoch [23/25], Loss: 0.2043
Epoch [24/25], Loss: 0.1616
Epoch [25/25], Loss: 0.1923


**Test - Inference**

In [12]:
import torch
from torchvision import transforms, models
from PIL import Image

**Loading pre-trained model**

In [13]:
model_path = '/kaggle/working/logo_classification_model_epoch25.pth'

# Load the model
model = models.resnet50(pretrained=False)
num_classes = 2  
in_features = model.fc.in_features
model.fc = torch.nn.Linear(in_features, num_classes)
model.load_state_dict(torch.load(model_path))
model.eval()

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

**Set the device for inference and apply transformation**

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

image_transforms = transforms.Compose([
    transforms.Resize((70, 70)),
    transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                          std=[0.229, 0.224, 0.225])
])

**Function to perform inference on a single image**

In [15]:
def classify_logo(image_path):
    image = Image.open(image_path).convert('RGB')
    image = image_transforms(image).unsqueeze(0)
    image = image.to(device)

    with torch.no_grad():
        outputs = model(image)

    _, predicted = torch.max(outputs.data, 1)
    class_index = predicted.item()
    classes = ['genuine', 'fake']
    class_label = classes[class_index]

    return class_label

**Example Usage**

In [16]:
test_image_path = '/kaggle/input/test-image/download.png'
predicted_class = classify_logo(test_image_path)
print("Predicted class:", predicted_class)

Predicted class: fake
